In [ ]:
import numpy as np

def rsm(decision_matrix, reference_targets, reference_nadir, weights=None):
    """
    RSM dla wariantu ciągłego i dyskretnego.

    :param decision_matrix: Macierz decyzyjna (alternatywy x kryteria).
    :param reference_targets: Punkt docelowy (1 x kryteria).
    :param reference_nadir: Punkt antyidealny (1 x kryteria).
    :param weights: Wektory wag kryteriów. Jeśli None, zakładamy równomierne wagi.
    :return: Ranking alternatyw i wartości wskaźnika RSM.
    """
    num_alternatives, num_criteria = decision_matrix.shape

    # Ustawienie wag
    if weights is None:
        weights = np.ones(num_criteria) / num_criteria

    # Normalizacja wag
    weights = weights / np.sum(weights)

    # Normalizacja macierzy decyzyjnej
    norm_matrix = decision_matrix / np.max(decision_matrix, axis=0)

    # Sprawdzanie wymiarów punktów odniesienia
    if len(reference_targets) != num_criteria or len(reference_nadir) != num_criteria:
        raise ValueError("Wymiary punktów odniesienia muszą odpowiadać liczbie kryteriów.")

    # Obliczenie odległości od punktów docelowych
    distances_to_targets = np.sqrt(
        np.sum(weights * (norm_matrix - reference_targets) ** 2, axis=1)
    )

    # Obliczenie odległości od punktów nadir
    distances_to_nadir = np.sqrt(
        np.sum(weights * (norm_matrix - reference_nadir) ** 2, axis=1)
    )

    # Wskaźnik RSM: różnica odległości
    rsm_scores = distances_to_nadir - distances_to_targets

    # Ranking alternatyw
    ranking = np.argsort(rsm_scores)[::-1]  # Malejąco według wskaźnika

    return ranking, rsm_scores

# Funkcja generująca dane dla wariantu ciągłego
def generate_continuous_data(n_alternatives, n_criteria, ranges):
    """
    Generowanie losowych danych ciągłych w przestrzeni decyzyjnej.
    """
    return np.random.uniform(low=ranges[:, 0], high=ranges[:, 1], size=(n_alternatives, n_criteria))

# Przykład wariantu ciągłego
n_alternatives = 10
n_criteria = 3
criteria_ranges = np.array([[0, 1], [0, 100], [0, 50]])  # Zakres dla każdego kryterium
decision_matrix_continuous = generate_continuous_data(n_alternatives, n_criteria, criteria_ranges)

# Punkty odniesienia dla ciągłego
reference_targets_continuous = np.array([0.8, 80, 40])  # Punkt docelowy (3 kryteria)
reference_nadir_continuous = np.array([0.2, 20, 10])    # Punkt antyidealny (3 kryteria)
weights = np.array([0.4, 0.3, 0.3])  # Wagi kryteriów

# Wywołanie RSM dla wariantu ciągłego
ranking_rsm_continuous, scores_rsm_continuous = rsm(
    decision_matrix_continuous, reference_targets_continuous, reference_nadir_continuous, weights
)
print("Ranking RSM (ciągły):", ranking_rsm_continuous + 1)
print("Wartości wskaźnika RSM (ciągły):", scores_rsm_continuous)

# Przykład wariantu dyskretnego
decision_matrix_discrete = np.array([
    [250, 16, 12],
    [200, 20, 15],
    [300, 12, 10]
])
reference_targets_discrete = np.array([300, 20, 10])  # Punkt docelowy (3 kryteria)
reference_nadir_discrete = np.array([200, 12, 15])    # Punkt antyidealny (3 kryteria)

# Wywołanie RSM dla wariantu dyskretnego
ranking_rsm_discrete, scores_rsm_discrete = rsm(
    decision_matrix_discrete, reference_targets_discrete, reference_nadir_discrete, weights
)
print("Ranking RSM (dyskretny):", ranking_rsm_discrete + 1)
print("Wartości wskaźnika RSM (dyskretny):", scores_rsm_discrete)

In [ ]:

import numpy as np

def fuzzy_topsis(decision_matrix, weights, criteria_types):
    """
    Fuzzy TOPSIS dla wariantu ciągłego.

    :param decision_matrix: Macierz decyzyjna jako trójkątne liczby rozmyte [(l, m, u)].
    :param weights: Wagi kryteriów jako trójkątne liczby rozmyte [(l, m, u)].
    :param criteria_types: Typy kryteriów ("max" dla maksymalizacji, "min" dla minimalizacji).
    :return: Ranking alternatyw i odległości od FPIS/FNIS.
    """
    num_alternatives, num_criteria, _ = decision_matrix.shape

    # Normalizacja trójkątnych liczb rozmytych
    normalized_matrix = np.zeros_like(decision_matrix)
    for j in range(num_criteria):
        if criteria_types[j] == "max":
            max_u = np.max(decision_matrix[:, j, 2])  # Najwyższa wartość górna (u)
            normalized_matrix[:, j, :] = decision_matrix[:, j, :] / max_u
        else:  # criteria_types[j] == "min"
            min_l = np.min(decision_matrix[:, j, 0])  # Najniższa wartość dolna (l)
            normalized_matrix[:, j, :] = min_l / decision_matrix[:, j, :]

    # Uwzględnienie wag
    weighted_matrix = np.zeros_like(normalized_matrix)
    for j in range(num_criteria):
        weighted_matrix[:, j, :] = normalized_matrix[:, j, :] * weights[j]

    # Wyznaczenie punktów FPIS i FNIS
    fpis = np.max(weighted_matrix, axis=0)  # Idealny punkt rozmyty (l*, m*, u*)
    fnis = np.min(weighted_matrix, axis=0)  # Antyidealny punkt rozmyty (l_*, m_*, u_)

    # Obliczenie odległości od FPIS i FNIS
    distances_to_fpis = np.sqrt(np.sum((weighted_matrix - fpis) ** 2, axis=(1, 2)))
    distances_to_fnis = np.sqrt(np.sum((weighted_matrix - fnis) ** 2, axis=(1, 2)))

    # Relatywna bliskość do FPIS
    scores = distances_to_fnis / (distances_to_fpis + distances_to_fnis)

    # Ranking
    ranking = np.argsort(scores)[::-1]  # Malejący porządek

    return ranking, scores

# Generowanie danych dla wariantu ciągłego
def generate_fuzzy_continuous_data(n_alternatives, n_criteria, ranges):
    """
    Generowanie danych wejściowych jako trójkątne liczby rozmyte dla wariantu ciągłego.
    """
    decision_matrix = np.zeros((n_alternatives, n_criteria, 3))
    for j in range(n_criteria):
        l = np.random.uniform(ranges[j, 0], ranges[j, 1], size=n_alternatives)
        m = l + np.random.uniform(0, (ranges[j, 1] - ranges[j, 0]) / 2, size=n_alternatives)
        u = m + np.random.uniform(0, (ranges[j, 1] - ranges[j, 0]) / 2, size=n_alternatives)
        decision_matrix[:, j, 0] = l
        decision_matrix[:, j, 1] = m
        decision_matrix[:, j, 2] = u
    return decision_matrix

# Dane wejściowe
n_alternatives = 5
n_criteria = 3
criteria_ranges = np.array([[0, 10], [0, 50], [0, 100]])  # Zakresy dla kryteriów
decision_matrix_continuous = generate_fuzzy_continuous_data(n_alternatives, n_criteria, criteria_ranges)

weights = np.array([[0.3, 0.5, 0.7], [0.2, 0.4, 0.6], [0.4, 0.6, 0.8]])  # Wagi jako liczby rozmyte
criteria_types = ["max", "max", "min"]

# Wywołanie Fuzzy TOPSIS
ranking, scores = fuzzy_topsis(decision_matrix_continuous, weights, criteria_types)

print("Ranking (Fuzzy TOPSIS):", ranking + 1)
print("Wartości wskaźników (Fuzzy TOPSIS):", scores)

In [ ]:

import numpy as np

def uta(decision_matrix, num_segments, weights=None):
    """
    Implementacja metody UTA (Utilité Additive) dla wariantu ciągłego i dyskretnego.

    :param decision_matrix: Macierz decyzyjna (alternatywy x kryteria).
    :param num_segments: Liczba segmentów dla każdej funkcji użyteczności.
    :param weights: Wagi kryteriów. Jeśli None, zakładamy równomierne wagi.
    :return: Użyteczności alternatyw, ranking, funkcje użyteczności dla kryteriów.
    """
    num_alternatives, num_criteria = decision_matrix.shape

    # Normalizacja macierzy decyzyjnej
    norm_matrix = (decision_matrix - decision_matrix.min(axis=0)) / (
        decision_matrix.max(axis=0) - decision_matrix.min(axis=0)
    )

    # Inicjalizacja wag
    if weights is None:
        weights = np.ones(num_criteria) / num_criteria

    # Tworzenie funkcji użyteczności dla każdego kryterium
    utility_functions = np.zeros((num_segments + 1, num_criteria))
    for crit in range(num_criteria):
        # Liniowa funkcja użyteczności w segmentach [0, 1]
        utility_functions[:, crit] = np.linspace(0, 1, num_segments + 1)

    # Wyznaczenie użyteczności alternatyw
    utilities = np.zeros(num_alternatives)
    for i in range(num_alternatives):
        for crit in range(num_criteria):
            value = norm_matrix[i, crit]
            # Znalezienie odpowiedniego segmentu
            segment_index = int(value * num_segments)
            segment_index = min(segment_index, num_segments - 1)  # Obsługa granicy
            utilities[i] += weights[crit] * utility_functions[segment_index, crit]

    # Ranking alternatyw
    ranking = np.argsort(-utilities)  # Malejący porządek użyteczności

    return utilities, ranking, utility_functions


# Przykład: Wariant ciągły
def generate_continuous_data(n_alternatives, n_criteria, ranges):
    """
    Generowanie danych dla wariantu ciągłego.
    """
    return np.random.uniform(low=ranges[:, 0], high=ranges[:, 1], size=(n_alternatives, n_criteria))

# Dane dla wariantu ciągłego
n_alternatives = 10
n_criteria = 3
criteria_ranges = np.array([[0, 1], [0, 100], [0, 50]])  # Zakresy wartości kryteriów
decision_matrix_continuous = generate_continuous_data(n_alternatives, n_criteria, criteria_ranges)

# Liczba segmentów i wagi
num_segments = 5
weights = np.array([0.4, 0.3, 0.3])  # Wagi kryteriów

# Wywołanie dla wariantu ciągłego
utilities_cont, ranking_cont, utility_functions_cont = uta(decision_matrix_continuous, num_segments, weights)
print("Użyteczności (ciągły):", utilities_cont)
print("Ranking (ciągły):", ranking_cont + 1)

# Dane dla wariantu dyskretnego
decision_matrix_discrete = np.array([
    [250, 16, 12],
    [200, 20, 15],
    [300, 12, 10]
])

# Wywołanie dla wariantu dyskretnego
utilities_disc, ranking_disc, utility_functions_disc = uta(decision_matrix_discrete, num_segments, weights)
print("Użyteczności (dyskretny):", utilities_disc)
print("Ranking (dyskretny):", ranking_disc + 1)

In [ ]:


import numpy as np

def topsis(decision_matrix, weights, criteria_types):
    """
    Implementacja algorytmu TOPSIS.

    :param decision_matrix: Macierz decyzyjna (alternatywy x kryteria).
    :param weights: Wektory wag kryteriów.
    :param criteria_types: Typy kryteriów ("max" dla maksymalizacji, "min" dla minimalizacji).
    :return: Ranking alternatyw.
    """
    # Krok 1: Normalizacja macierzy decyzyjnej
    norm_matrix = decision_matrix / np.sqrt(np.sum(decision_matrix**2, axis=0))

    # Krok 2: Uwzględnienie wag
    weighted_matrix = norm_matrix * weights

    # Krok 3: Wyznaczenie punktów idealnych i antyidealnych
    ideal_solution = []
    anti_ideal_solution = []

    for j, crit_type in enumerate(criteria_types):
        if crit_type == "max":
            ideal_solution.append(np.max(weighted_matrix[:, j]))
            anti_ideal_solution.append(np.min(weighted_matrix[:, j]))
        else:  # crit_type == "min"
            ideal_solution.append(np.min(weighted_matrix[:, j]))
            anti_ideal_solution.append(np.max(weighted_matrix[:, j]))

    ideal_solution = np.array(ideal_solution)
    anti_ideal_solution = np.array(anti_ideal_solution)

    # Krok 4: Obliczenie odległości od rozwiązań idealnych
    distances_to_ideal = np.sqrt(np.sum((weighted_matrix - ideal_solution)**2, axis=1))
    distances_to_anti_ideal = np.sqrt(np.sum((weighted_matrix - anti_ideal_solution)**2, axis=1))

    # Krok 5: Obliczenie wskaźnika skoringowego
    scores = distances_to_anti_ideal / (distances_to_ideal + distances_to_anti_ideal)

    # Krok 6: Ranking
    ranking = np.argsort(scores)[::-1]  # Sortowanie malejące według wskaźnika

    return ranking, scores

# Przykładowe dane
decision_matrix = np.array([
    [250, 16, 12],
    [200, 20, 15],
    [300, 12, 10]
])

weights = np.array([0.5, 0.3, 0.2])
criteria_types = ["max", "max", "min"]

ranking, scores = topsis(decision_matrix, weights, criteria_types)

print("Ranking alternatyw:", ranking + 1)  # Indeksy alternatyw (od 1)
print("Wartości wskaźnika TOPSIS:", scores)